Ссылка на БД: https://tochno.st/datasets/bdmo


In [ ]:
%%capture
!pip install osmnx pyrosm
!apt-get update -qq && apt-get install -y osmium-tool

In [ ]:
import pandas as pd
import numpy as np
import json
from pyrosm import OSM, get_data
import requests
from shapely.geometry import LineString
from shapely.ops import polygonize, unary_union
import json
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Население (данные 2023 года)

Файл: data_Y48112015_year2023_112_v20241111



In [ ]:
# Площадь каждого округа в гектарах (спарсил ручками)

districts_area = {
    # центральный
    "Арбат": 288,
    "Басманный": 837,
    "Замоскворечье": 495,
    "Красносельский": 496,
    "Мещанский": 576,
    "Пресненский": 1185,
    "Таганский": 801,
    "Тверской": 727,
    "Хамовники": 1192,
    "Якиманка": 480,

    # северный
    "Аэропорт": 458,
    "Беговой": 556,
    "Бескудниковский": 330,
    "Войковский": 661,
    "Восточное Дегунино": 377,
    "Головинский": 893,
    "Дмитровский": 729,
    "Западное Дегунино": 753,
    "Коптево": 538,
    "Левобережный": 646,
    "Молжаниновский": 2178,
    "Савеловский": 270,
    "Сокол": 372,
    "Тимирязевский": 1043,
    "Ховрино": 573,
    "Хорошевский": 988,

    # северно-восточный
    "Алексеевский": 529,
    "Алтуфьевский": 325,
    "Бабушкинский": 507,
    "Бибирево": 645,
    "Бутырский": 504,
    "Лианозово": 579,
    "Лосиноостровский": 554,
    "Марфино": 226,
    "Марьина роща": 468,
    "Останкинский": 1246,
    "Отрадное": 1016,
    "Ростокино": 354,
    "Свиблово": 441,
    "Северное Медведково": 566,
    "Южное Медведково": 1029,
    "Ярославский": 387,
    "Северный": 1500,

    # восточный
    "Богородское": 865,
    "Вешняки": 1067,
    "Восточное Измайлово": 385,
    "Восточный": 320,
    "Гольяново": 1499,
    "Ивановское": 1017,
    "Измайлово": 1524,
    "Косино-Ухтомский": 1506,
    "Метрогородок": 2757,
    "Новогиреево": 446,
    "Новокосино": 360,
    "Перово": 973,
    "Преображенское": 562,
    "Северное Измайлово": 420,
    "Соколиная гора": 784,
    "Сокольники": 1028,

    # юго-восточный
    "Выхино-Жулебино": 1497,
    "Капотня": 806,
    "Кузьминки": 815,
    "Лефортово": 906,
    "Люблино": 1741,
    "Марьино": 1198,
    "Некрасовка": 1147,
    "Нижегородский": 753,
    "Печатники": 1789,
    "Рязанский": 649,
    "Текстильщики": 591,
    "Южнопортовый": 453,

    # южный
    "Бирюлево Восточное": 1477.2,
    "Бирюлево Западное": 850.64,
    "Братеево": 763.0,
    "Даниловский": 1259.6,
    "Донской": 572.85,
    "Зябликово": 437.95,
    "Москворечье-Сабурово": 930.15,
    "Нагатино-Садовники": 817.0,
    "Нагатинский затон": 979.5,
    "Нагорный": 541.61,
    "Орехово-Борисово Северное": 767.16,
    "Орехово-Борисово Южное": 694.26,
    "Царицыно": 843.44,
    "Чертаново Северное": 540.16,
    "Чертаново Центральное": 651.59,
    "Чертаново Южное": 938.0,

    # юго-западный
    "Академический": 583.44,
    "Гагаринский": 549.91,
    "Зюзино": 545.04,
    "Коньково": 717.91,
    "Котловка": 394.41,
    "Ломоносовский": 333.75,
    "Обручевский": 610.94,
    "Северное Бутово": 913.38,
    "Теплый Стан": 750.0,
    "Черемушки": 551.86,
    "Южное Бутово": 2553.74,
    "Ясенево": 2536.65,

    # западный
    "Дорогомилово": 795.0,
    "Крылатское": 1204.46,
    "Кунцево": 1656.48,
    "Можайский": 1072.6,
    "Ново-Переделкино": 847.82,
    "Очаково-Матвеевское": 1754.36,
    "Проспект Вернадского": 465.14,
    "Раменки": 1853.71,
    "Солнцево": 1129.0,
    "Тропарево-Никулино": 1126.66,
    "Филевский парк": 962.43,
    "Фили-Давыдково": 695.71,

    # северо-западный
    "Куркино": 804,
    "Митино": 1266,
    "Покровское-Стрешнево": 1290,
    "Северное Тушино": 940,
    "Строгино": 1684,
    "Хорошево-Мневники": 1781,
    "Щукино": 768,
    "Южное Тушино": 794,

    # зеленоградский
    "Матушкино": 503.1,
    "Савелки": 812.2,
    "Старое Крюково": 380.5,
    "Силино": 1039.63,
    "Крюково": 1125.0,

    # троицкий (границы 2012-2024)
    "Бекасово": 21096.56,
    "Вороновское": 51392.88,
    "Краснопахорское": 25157.90,
    "Троицк": 9851.92,
    "Киевский": 6009,
    "Кленовское": 11772,
    "Михайлово-Ярцевское": 6461,
    "Новофедоровское": 15080,
    "Первомайское": 11892,
    "Роговское": 17818,
    "Щаповское": 8688,

    # новомосковский (границы 2012-2024)
    "Внуковское": 2559,
    "Внуково": 1691,
    "Воскресенское": 2482,
    "Десеновское": 5300,
    "Московский": 4039,
    "Щербинка": 6530.60,
    "Кокошкино": 900,
    "Марушкинское": 5063,
    '«Мосрентген»': 648,
    "Рязановское": 4048,
    "Сосенское": 6681,
    "Филимонковское": 3572
}

In [ ]:
## Плотность всего населения
## Численность городского населения по полу и возрасту на 1 января текущего года
file_path = "drive/MyDrive/masters/geo-embeddings/data/popul_age_stats.csv"

df = pd.read_csv(file_path, sep=';')
df.head(2)

<ipython-input-5-d211bbc18259>:5: DtypeWarning: Columns (14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';')


,indicator_section_code,indicator_section,indicator_code,indicator_name,grup_2,vozr,region_id,region_name,mun_level,mun_district,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,indicator_value,indicator_unit,indicator_period,comment
0,31,Население,Y48112015,Численность городского населения по полу и воз...,Всего,5,1,Алтайский край,Муниципальное образование верхнего уровня,Муниципальный округ город Славгород,...,Муниципальный округ,1539000,"Изменен тип и название, Объединение",2023,2025,2023,252.0,Человек,На 1 января,CD
1,31,Население,Y48112015,Численность городского населения по полу и воз...,Мужчины,26,1,Алтайский край,Муниципальное образование верхнего уровня,Муниципальный округ город Славгород,...,Муниципальный округ,1539000,"Изменен тип и название, Объединение",2023,2025,2023,117.0,Человек,На 1 января,CD


In [ ]:
# Фильтруем Москву
df = df[df["region_name"] == "Москва"]

# Убираем хвостики у названий
patterns = ['Муниципальный округ ', 'Городской округ ', 'Поселение ']
for pattern in patterns:
        df['mun_district'] = df['mun_district'].str.replace(pattern, '')

# Кол-во муниципалитетов, округов и прочего
mun_num = len(df["mun_district"].unique())

print(f"Москва, 132 муниципалитета и 12 округов: {mun_num}")

Москва, 132 муниципалитета и 12 округов: 147


In [ ]:
# Добавим площадь
df['area'] = df['mun_district'].apply(
    lambda x: districts_area.get(x, None)
    )

# Переведем га в км
df['area'] = df['area'] / 100

In [ ]:
# Меняем категорию, чтобы система видела пожилых (пусть 72)
df["vozr"] = df["vozr"].replace("70 и старше", "72")

df["grup_2"].unique(), df["vozr"].unique()

(array(['Женщины', 'Всего', 'Мужчины'], dtype=object),
 array(['Всего', '66', '5‒18', '14‒15', '5', '28', '0‒14', '35', '3',
        '30‒34', '60', '20', '16‒29', '31', '57', '39', '20‒24', '46',
        '0‒4', '25‒29', 'До 30', '27', '8‒13', '4', '61', '45', '55‒59',
        'Старше 65', '40‒44', '50‒59', '10‒14', '21', '67', '52', '55',
        '40', '32', '17', '36', '60‒64', '16', '25', '38', '62', '30',
        '5‒17', '15‒49', '41', '0‒2', '45‒49', '10', '15', '51', '42', '1',
        '49', '0‒17', '2', '72', '13', '54', '69', '35‒39', '34', '56',
        '59', '1‒6', '30‒39', 'Старше трудоспособного возраста', '53', '7',
        '43', '68', '40‒49', '44', 'Трудоспособный возраст', '8', '18',
        '19', '50‒54', '22', '33', '60‒65', '11', '14', '47', '65', '50',
        '3‒5', '23', '6', '5‒9', '63', '16‒17', '12', '65‒69', '37', '24',
        '9', '48', '29', 'Моложе трудоспособного возраста', '15‒19',
        '18‒19', '58', '26', '0', '64'], dtype=object))

In [ ]:
df_pop = df[(df["grup_2"] == "Всего") & (df["vozr"] == "Всего")]
df_pop = df_pop.rename(columns={"indicator_value": "population"})
df_pop.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,grup_2,vozr,region_id,region_name,mun_level,mun_district,...,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,population,indicator_unit,indicator_period,comment,area
299004,31,Население,Y48112015,Численность городского населения по полу и воз...,Всего,Всего,45,Москва,Субъект РФ,Муниципальные образования города Москвы (столи...,...,CD,CD,CD,CD,2023,13104177.0,Человек,На 1 января,mun_type в исходных данных не соответствует oktmo,NaN
299295,31,Население,Y48112015,Численность городского населения по полу и воз...,Всего,Всего,45,Москва,Муниципальное образование верхнего уровня,Богородское,...,45301000,Изменен тип и название,2010,2013,2023,108657.0,Человек,На 1 января,CD,8.65


### Плотность всего населения

По муниципальным округам (чел/км2)

In [ ]:
# Плотность всего населения по округам
filtered_data = df[(df["grup_2"] == "Всего") & (df["vozr"] == "Всего")]

filtered_data["pop_dense"] = filtered_data["indicator_value"] / filtered_data["area"]
filtered_data[["mun_district", "indicator_value", "area", "pop_dense"]]

<ipython-input-10-256b7e7c504b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["pop_dense"] = filtered_data["indicator_value"] / filtered_data["area"]


,mun_district,indicator_value,area,pop_dense
299004,Муниципальные образования города Москвы (столи...,13104177.0,NaN,NaN
299295,Богородское,108657.0,8.65,12561.502890
299617,Вешняки,119927.0,10.67,11239.643861
300117,Восточное Измайлово,73929.0,3.85,19202.337662
300463,Восточный,12906.0,3.20,4033.125000
...,...,...,...,...
345040,Роговское,6478.0,178.18,36.356493
345454,Рязановское,32215.0,40.48,795.825099
345637,Сосенское,115545.0,66.81,1729.456668
346106,Филимонковское,16237.0,35.72,454.563270


In [ ]:
# Проверка, что захватили все округа
filtered_data[filtered_data["area"].isna()]

,indicator_section_code,indicator_section,indicator_code,indicator_name,grup_2,vozr,region_id,region_name,mun_level,mun_district,...,oktmo_history,oktmo_year_from,oktmo_year_to,year,indicator_value,indicator_unit,indicator_period,comment,area,pop_dense
299004,31,Население,Y48112015,Численность городского населения по полу и воз...,Всего,Всего,45,Москва,Субъект РФ,Муниципальные образования города Москвы (столи...,...,CD,CD,CD,2023,13104177.0,Человек,На 1 января,mun_type в исходных данных не соответствует oktmo,NaN,NaN


In [ ]:
# Начинаем составлять итоговый датасет
result = filtered_data[["mun_district", "indicator_value", "area", "pop_dense"]].reset_index()
result = result.drop("index", axis=1)

### Плотность трудоспособного населения

По муниципальным округам (чел/км2)

In [ ]:
# Плотность трудоспособного населения по округам
filtered_data = df[(df["grup_2"] == "Всего") & (df["vozr"] == "Трудоспособный возраст")]

filtered_data["pop_work_dense"] = filtered_data["indicator_value"] / filtered_data["area"]
filtered_data = filtered_data[["mun_district", "indicator_value", "area", "pop_work_dense"]]
filtered_data

<ipython-input-13-39ba280243eb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["pop_work_dense"] = filtered_data["indicator_value"] / filtered_data["area"]


,mun_district,indicator_value,area,pop_work_dense
299027,Муниципальные образования города Москвы (столи...,7512670.0,NaN,NaN
299488,Богородское,59738.0,8.65,6906.127168
299825,Вешняки,68282.0,10.67,6399.437676
299969,Восточное Измайлово,44275.0,3.85,11500.000000
300421,Восточный,7461.0,3.20,2331.562500
...,...,...,...,...
345234,Роговское,3307.0,178.18,18.559883
345408,Рязановское,18543.0,40.48,458.078063
345686,Сосенское,65912.0,66.81,986.558898
346189,Филимонковское,8759.0,35.72,245.212766


In [ ]:
# Продолжаем составлять итоговый датасет
result = pd.merge(result, filtered_data[["mun_district", "pop_work_dense"]], on="mun_district", how="left")
result

,mun_district,indicator_value,area,pop_dense,pop_work_dense
0,Муниципальные образования города Москвы (столи...,13104177.0,NaN,NaN,NaN
1,Богородское,108657.0,8.65,12561.502890,6906.127168
2,Вешняки,119927.0,10.67,11239.643861,6399.437676
3,Восточное Измайлово,73929.0,3.85,19202.337662,11500.000000
4,Восточный,12906.0,3.20,4033.125000,2331.562500
...,...,...,...,...,...
142,Роговское,6478.0,178.18,36.356493,18.559883
143,Рязановское,32215.0,40.48,795.825099,458.078063
144,Сосенское,115545.0,66.81,1729.456668,986.558898
145,Филимонковское,16237.0,35.72,454.563270,245.212766


### Плотность детей (ниже труд. возр)

По муниципальным округам (чел/км2)

In [ ]:
# Плотность детей по округам
filtered_data = df[(df["grup_2"] == "Всего") & (df["vozr"] == "Моложе трудоспособного возраста")]

filtered_data["pop_child_dense"] = filtered_data["indicator_value"] / filtered_data["area"]
filtered_data = filtered_data[["mun_district", "indicator_value", "area", "pop_child_dense"]]
filtered_data

<ipython-input-15-e7e4951d18a8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["pop_child_dense"] = filtered_data["indicator_value"] / filtered_data["area"]


,mun_district,indicator_value,area,pop_child_dense
299129,Муниципальные образования города Москвы (столи...,2073639.0,NaN,NaN
299470,Богородское,17270.0,8.65,1996.531792
299576,Вешняки,16646.0,10.67,1560.074977
300067,Восточное Измайлово,11980.0,3.85,3111.688312
300373,Восточный,1663.0,3.20,519.687500
...,...,...,...,...
345030,Роговское,1112.0,178.18,6.240880
345350,Рязановское,5593.0,40.48,138.166996
345748,Сосенское,26649.0,66.81,398.877414
346001,Филимонковское,3571.0,35.72,99.972004


In [ ]:
# Продолжаем составлять итоговый датасет
result = pd.merge(result, filtered_data[["mun_district", "pop_child_dense"]], on="mun_district", how="left")
result

,mun_district,indicator_value,area,pop_dense,pop_work_dense,pop_child_dense
0,Муниципальные образования города Москвы (столи...,13104177.0,NaN,NaN,NaN,NaN
1,Богородское,108657.0,8.65,12561.502890,6906.127168,1996.531792
2,Вешняки,119927.0,10.67,11239.643861,6399.437676,1560.074977
3,Восточное Измайлово,73929.0,3.85,19202.337662,11500.000000,3111.688312
4,Восточный,12906.0,3.20,4033.125000,2331.562500,519.687500
...,...,...,...,...,...,...
142,Роговское,6478.0,178.18,36.356493,18.559883,6.240880
143,Рязановское,32215.0,40.48,795.825099,458.078063,138.166996
144,Сосенское,115545.0,66.81,1729.456668,986.558898,398.877414
145,Филимонковское,16237.0,35.72,454.563270,245.212766,99.972004


### Средний возраст

По муниципальным округам

In [ ]:
# Средний возраст
avg_age = {}

filtered_data = df[(df["grup_2"] == "Всего")]
for mun in filtered_data["mun_district"].unique():
    sum = 0
    delim = 0
    mun_data = filtered_data[filtered_data["mun_district"] == mun]
    for _, row in mun_data.iterrows():
        try:
            age_group = int(row["vozr"])
            sum += age_group * row["indicator_value"]
            delim += row["indicator_value"]
        except:
            continue
    avg_age[mun] = sum / delim

In [ ]:
# Продолжаем составлять итоговый датасет
result["avg_age"] = result["mun_district"].apply(lambda x: avg_age[x])
result

,mun_district,indicator_value,area,pop_dense,pop_work_dense,pop_child_dense,avg_age
0,Муниципальные образования города Москвы (столи...,13104177.0,NaN,NaN,NaN,NaN,41.300748
1,Богородское,108657.0,8.65,12561.502890,6906.127168,1996.531792,41.819579
2,Вешняки,119927.0,10.67,11239.643861,6399.437676,1560.074977,43.079131
3,Восточное Измайлово,73929.0,3.85,19202.337662,11500.000000,3111.688312,39.658564
4,Восточный,12906.0,3.20,4033.125000,2331.562500,519.687500,42.835735
...,...,...,...,...,...,...,...
142,Роговское,6478.0,178.18,36.356493,18.559883,6.240880,43.121179
143,Рязановское,32215.0,40.48,795.825099,458.078063,138.166996,41.049666
144,Сосенское,115545.0,66.81,1729.456668,986.558898,398.877414,36.707015
145,Филимонковское,16237.0,35.72,454.563270,245.212766,99.972004,38.732463


## Розничная торговля и общественное питание

Файл: data_Y48201003_112_v20241111

- Оборот розничной торговли (без субъектов малого предпринимательства) на 1 чел

In [ ]:
## Оборот торговли
file_path = "drive/MyDrive/masters/geo-embeddings/data/retail_trade_stats.csv"

df = pd.read_csv(file_path)
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,okved,region_id,region_name,mun_level,mun_district,municipality,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,indicator_value,indicator_unit,indicator_period,comment
0,2,Розничная торговля и общественное питание,Y48201003,Оборот розничной торговли (без субъектов малог...,Розничная торговля в неспециализированных мага...,1,Алтайский край,Муниципальное образование верхнего уровня,Алейский муниципальный район,Алейский муниципальный район,...,Муниципальный район,01601000,Без изменений,2010,2025,2011,24822.0,Тысяча рублей,Январь-сентябрь,CD
1,2,Розничная торговля и общественное питание,Y48201003,Оборот розничной торговли (без субъектов малог...,"Розничная торговля, кроме торговли автотранспо...",1,Алтайский край,Муниципальное образование верхнего уровня,Алейский муниципальный район,Алейский муниципальный район,...,Муниципальный район,01601000,Без изменений,2010,2025,2011,24822.0,Тысяча рублей,Январь-сентябрь,CD


In [ ]:
# Фильтруем Москву, годовой показатель и последний доступный год
df = df[(df["region_name"] == "Москва") &
        (df["indicator_period"] == "Январь-декабрь") &
        (df["year"] == 2016) &
        (df["okved"] == "Всего")]

# Убираем хвостики у названий
patterns = ['Муниципальный округ ', 'Городской округ ', 'Поселение ']
for pattern in patterns:
        df['mun_district'] = df['mun_district'].str.replace(pattern, '')

# Кол-во муниципалитетов, округов и прочего
mun_num = len(df["mun_district"].unique())

print(f"Москва, 132 муниципалитета и 12 округов: {mun_num}")

Москва, 132 муниципалитета и 12 округов: 89


<ipython-input-20-573a301a2698>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mun_district'] = df['mun_district'].str.replace(pattern, '')


Очевидно, не хватает большого кол-ва данных, да и сами цифры старые, аж 2016 года, стоит подумать, нужны ли они нам

In [ ]:
df_pop.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,grup_2,vozr,region_id,region_name,mun_level,mun_district,...,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,population,indicator_unit,indicator_period,comment,area
299004,31,Население,Y48112015,Численность городского населения по полу и воз...,Всего,Всего,45,Москва,Субъект РФ,Муниципальные образования города Москвы (столи...,...,CD,CD,CD,CD,2023,13104177.0,Человек,На 1 января,mun_type в исходных данных не соответствует oktmo,NaN
299295,31,Население,Y48112015,Численность городского населения по полу и воз...,Всего,Всего,45,Москва,Муниципальное образование верхнего уровня,Богородское,...,45301000,Изменен тип и название,2010,2013,2023,108657.0,Человек,На 1 января,CD,8.65


In [ ]:
# Считаем относит. розн. торговлю на 1 чел
df = pd.merge(df, df_pop[["mun_district", "population"]], on='mun_district', how='left')
df["trade_per_pers"] = df["indicator_value"] / df["population"]
df.head(5)

,indicator_section_code,indicator_section,indicator_code,indicator_name,okved,region_id,region_name,mun_level,mun_district,municipality,...,oktmo_history,oktmo_year_from,oktmo_year_to,year,indicator_value,indicator_unit,indicator_period,comment,population,trade_per_pers
0,2,Розничная торговля и общественное питание,Y48201003,Оборот розничной торговли (без субъектов малог...,Всего,45,Москва,Муниципальное образование верхнего уровня,Богородское,Муниципальный округ Богородское,...,Изменен тип и название,2010,2013,2016,397809.0,Тысяча рублей,Январь-декабрь,CD,108657.0,3.661145
1,2,Розничная торговля и общественное питание,Y48201003,Оборот розничной торговли (без субъектов малог...,Всего,45,Москва,Муниципальное образование верхнего уровня,Гольяново,Муниципальный округ Гольяново,...,Изменен тип и название,2010,2013,2016,16966173.3,Тысяча рублей,Январь-декабрь,CD,165243.0,102.674082
2,2,Розничная торговля и общественное питание,Y48201003,Оборот розничной торговли (без субъектов малог...,Всего,45,Москва,Муниципальное образование верхнего уровня,Измайлово,Муниципальный округ Измайлово,...,Изменен тип и название,2010,2013,2016,2890268.9,Тысяча рублей,Январь-декабрь,CD,107953.0,26.773400
3,2,Розничная торговля и общественное питание,Y48201003,Оборот розничной торговли (без субъектов малог...,Всего,45,Москва,Муниципальное образование верхнего уровня,Перово,Муниципальный округ Перово,...,Изменен тип и название,2010,2013,2016,1489381.9,Тысяча рублей,Январь-декабрь,CD,135315.0,11.006776
4,2,Розничная торговля и общественное питание,Y48201003,Оборот розничной торговли (без субъектов малог...,Всего,45,Москва,Муниципальное образование верхнего уровня,Северное Измайлово,Муниципальный округ Северное Измайлово,...,Изменен тип и название,2010,2013,2016,424548.3,Тысяча рублей,Январь-декабрь,CD,84986.0,4.995509


In [ ]:
# Продолжаем составлять итоговый датасет
result = pd.merge(result, df[["mun_district", "trade_per_pers"]], on="mun_district", how="left")
result

,mun_district,indicator_value,area,pop_dense,pop_work_dense,pop_child_dense,avg_age,trade_per_pers
0,Муниципальные образования города Москвы (столи...,13104177.0,NaN,NaN,NaN,NaN,41.300748,NaN
1,Богородское,108657.0,8.65,12561.502890,6906.127168,1996.531792,41.819579,3.661145
2,Вешняки,119927.0,10.67,11239.643861,6399.437676,1560.074977,43.079131,NaN
3,Восточное Измайлово,73929.0,3.85,19202.337662,11500.000000,3111.688312,39.658564,NaN
4,Восточный,12906.0,3.20,4033.125000,2331.562500,519.687500,42.835735,NaN
...,...,...,...,...,...,...,...,...
142,Роговское,6478.0,178.18,36.356493,18.559883,6.240880,43.121179,NaN
143,Рязановское,32215.0,40.48,795.825099,458.078063,138.166996,41.049666,NaN
144,Сосенское,115545.0,66.81,1729.456668,986.558898,398.877414,36.707015,NaN
145,Филимонковское,16237.0,35.72,454.563270,245.212766,99.972004,38.732463,NaN


## Здравоохранение

Файл: data_Y48018103_112_v20241111

- Население / Число больничных коек

In [ ]:
## Кол-во коек
file_path = "drive/MyDrive/masters/geo-embeddings/data/hosp_beds_stats.csv"

df = pd.read_csv(file_path)
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,zdrav,region_id,region_name,mun_level,mun_district,municipality,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,indicator_value,indicator_unit,indicator_period,comment
0,33,Здравоохранение,Y48018103,Число больничных коек,Самостоятельные больничные организации,1,Алтайский край,Муниципальное образование верхнего уровня,Алейский муниципальный район,Алейский муниципальный район,...,Муниципальный район,01601000,Без изменений,2010,2025,2008,75.0,Койка,Значение показателя за год,CD
1,33,Здравоохранение,Y48018103,Число больничных коек,Самостоятельные муниципальные больничные орган...,1,Алтайский край,Муниципальное образование верхнего уровня,Алейский муниципальный район,Алейский муниципальный район,...,Муниципальный район,01601000,Без изменений,2010,2025,2008,75.0,Койка,Значение показателя за год,CD


In [ ]:
# Фильтруем Москву и последний отчетный год
df = df[(df["region_name"] == "Москва") & (df["year"] == 2012)]

# Убираем хвостики у названий
patterns = ['Муниципальный округ ', 'Городской округ ', 'Поселение ']
for pattern in patterns:
        df['mun_district'] = df['mun_district'].str.replace(pattern, '')

# Кол-во муниципалитетов, округов и прочего
mun_num = len(df["mun_district"].unique())

print(f"Москва, 132 муниципалитета и 12 округов: {mun_num}")

Москва, 132 муниципалитета и 12 округов: 91


Очевидно, не хватает большого кол-ва данных, да и сами цифры старые, аж 2012 года, стоит подумать, нужны ли они нам

In [ ]:
# Есть разные виды больничек
df["zdrav"].unique()

array(['Самостоятельные больничные организации',
       'Негосударственные больничные организации',
       'Больничные отделения в составе ЦРБ и других ЛПО'], dtype=object)

In [ ]:
# Считаем общее кол-во коек во всех больничках
df = df.groupby('mun_district')['indicator_value'].sum()
df = df.reset_index()

In [ ]:
# Считаем относит. розн. торговлю на 1 чел
df = pd.merge(df, df_pop[["mun_district", "population"]], on='mun_district', how='left')
df["pers_per_bed"] = df["population"] / df["indicator_value"]
df.head(5)

,mun_district,indicator_value,population,pers_per_bed
0,Академический,830.0,113198.0,136.383133
1,Алексеевский,770.0,78502.0,101.950649
2,Арбат,562.0,35962.0,63.989324
3,Аэропорт,1020.0,81964.0,80.356863
4,Бабушкинский,1308.0,68075.0,52.045107


In [ ]:
# Продолжаем составлять итоговый датасет
result = pd.merge(result, df[["mun_district", "pers_per_bed"]], on="mun_district", how="left")
result

,mun_district,indicator_value,area,pop_dense,pop_work_dense,pop_child_dense,avg_age,trade_per_pers,pers_per_bed
0,Муниципальные образования города Москвы (столи...,13104177.0,NaN,NaN,NaN,NaN,41.300748,NaN,NaN
1,Богородское,108657.0,8.65,12561.502890,6906.127168,1996.531792,41.819579,3.661145,NaN
2,Вешняки,119927.0,10.67,11239.643861,6399.437676,1560.074977,43.079131,NaN,57.712705
3,Восточное Измайлово,73929.0,3.85,19202.337662,11500.000000,3111.688312,39.658564,NaN,18.088818
4,Восточный,12906.0,3.20,4033.125000,2331.562500,519.687500,42.835735,NaN,368.742857
...,...,...,...,...,...,...,...,...,...
142,Роговское,6478.0,178.18,36.356493,18.559883,6.240880,43.121179,NaN,NaN
143,Рязановское,32215.0,40.48,795.825099,458.078063,138.166996,41.049666,NaN,495.615385
144,Сосенское,115545.0,66.81,1729.456668,986.558898,398.877414,36.707015,NaN,3301.285714
145,Филимонковское,16237.0,35.72,454.563270,245.212766,99.972004,38.732463,NaN,NaN


## Деятельность предприятий

- Отгружено товаров собственного производства, выполнено работ и услуг собственными силами (без субъектов малого предпринимательства) - data_Y48401011_112_v20241111
- Продано товаров несобственного производства (без субъектов малого предпринимательства) - data_Y48201002_112_v20241111
- Количество организаций по данным государственной регистрации - data_Y49010002_112_v20241111
- Количество индивидуальных предпринимателей по данным государственной регистрации - data_Y49010003_112_v20241111

### Товары собственного производства

Файл: data_Y48401011_112_v20241111

In [ ]:
file_path = "drive/MyDrive/masters/geo-embeddings/data/self_goods_stats.csv"

df = pd.read_csv(file_path)
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,okved2,region_id,region_name,mun_level,mun_district,municipality,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,indicator_value,indicator_unit,indicator_period,comment
0,39,Деятельность предприятий,Y48401011,"Отгружено товаров собственного производства, в...",Всего по обследуемым видам экономической деяте...,1,Алтайский край,Муниципальное образование верхнего уровня,Залесовский муниципальный округ,Залесовский муниципальный округ,...,Муниципальный округ,01512000,Изменен тип и название,2022,2025,2022,244664.8,Тысяча рублей,Январь-декабрь,CD
1,39,Деятельность предприятий,Y48401011,"Отгружено товаров собственного производства, в...",Всего по обследуемым видам экономической деяте...,1,Алтайский край,Муниципальное образование верхнего уровня,Залесовский муниципальный округ,Залесовский муниципальный округ,...,Муниципальный округ,01512000,Изменен тип и название,2022,2025,2022,127158.5,Тысяча рублей,Январь-июнь,CD


In [ ]:
# Фильтруем Москву и последний отчетный год
df = df[(df["region_name"] == "Москва")
            & (df["year"] == 2023)
            & (df["okved2"] == "Всего по обследуемым видам экономической деятельности")
            & (df["indicator_period"] == "Январь-декабрь")]

# Убираем хвостики у названий
patterns = ['Муниципальный округ ', 'Городской округ ', 'Поселение ']
for pattern in patterns:
        df['mun_district'] = df['mun_district'].str.replace(pattern, '')

# Кол-во муниципалитетов, округов и прочего
mun_num = len(df["mun_district"].unique())

print(f"Москва, 132 муниципалитета и 12 округов: {mun_num}")

Москва, 132 муниципалитета и 12 округов: 144


In [ ]:
# Готово!
df = df.rename(columns={"indicator_value": "self_goods_sold"})
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,okved2,region_id,region_name,mun_level,mun_district,municipality,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,self_goods_sold,indicator_unit,indicator_period,comment
55970,39,Деятельность предприятий,Y48401011,"Отгружено товаров собственного производства, в...",Всего по обследуемым видам экономической деяте...,45,Москва,Муниципальное образование верхнего уровня,Богородское,Муниципальный округ Богородское,...,Внутригородская территория города федерального...,45301000,Изменен тип и название,2010,2013,2023,25493879.1,Тысяча рублей,Январь-декабрь,CD
55998,39,Деятельность предприятий,Y48401011,"Отгружено товаров собственного производства, в...",Всего по обследуемым видам экономической деяте...,45,Москва,Муниципальное образование верхнего уровня,Вешняки,Муниципальный округ Вешняки,...,Внутригородская территория города федерального...,45302000,Изменен тип и название,2010,2013,2023,22482322.9,Тысяча рублей,Январь-декабрь,CD


In [ ]:
# Продолжаем составлять итоговый датасет
result = pd.merge(result, df[["mun_district", "self_goods_sold"]], on="mun_district", how="left")
result

,mun_district,indicator_value,area,pop_dense,pop_work_dense,pop_child_dense,avg_age,trade_per_pers,pers_per_bed,self_goods_sold
0,Муниципальные образования города Москвы (столи...,13104177.0,NaN,NaN,NaN,NaN,41.300748,NaN,NaN,NaN
1,Богородское,108657.0,8.65,12561.502890,6906.127168,1996.531792,41.819579,3.661145,NaN,25493879.1
2,Вешняки,119927.0,10.67,11239.643861,6399.437676,1560.074977,43.079131,NaN,57.712705,22482322.9
3,Восточное Измайлово,73929.0,3.85,19202.337662,11500.000000,3111.688312,39.658564,NaN,18.088818,21708295.2
4,Восточный,12906.0,3.20,4033.125000,2331.562500,519.687500,42.835735,NaN,368.742857,69276.7
...,...,...,...,...,...,...,...,...,...,...
142,Роговское,6478.0,178.18,36.356493,18.559883,6.240880,43.121179,NaN,NaN,NaN
143,Рязановское,32215.0,40.48,795.825099,458.078063,138.166996,41.049666,NaN,495.615385,41010696.3
144,Сосенское,115545.0,66.81,1729.456668,986.558898,398.877414,36.707015,NaN,3301.285714,67726829.5
145,Филимонковское,16237.0,35.72,454.563270,245.212766,99.972004,38.732463,NaN,NaN,882499.8


### Товары несобственного производства

Файл: data_Y48201002_112_v20241111

In [ ]:
file_path = "drive/MyDrive/masters/geo-embeddings/data/for_goods_stats.csv"

df = pd.read_csv(file_path)
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,okved,region_id,region_name,mun_level,mun_district,municipality,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,indicator_value,indicator_unit,indicator_period,comment
0,39,Деятельность предприятий,Y48201002,Продано товаров несобственного производства (б...,Раздел Е Производство и распределение электроэ...,1,Алтайский край,Муниципальное образование верхнего уровня,Алейский муниципальный район,Алейский муниципальный район,...,Муниципальный район,01601000,Без изменений,2010,2025,2009,226205.5,Тысяча рублей,Январь-декабрь,CD
1,39,Деятельность предприятий,Y48201002,Продано товаров несобственного производства (б...,Раздел Е Производство и распределение электроэ...,1,Алтайский край,Муниципальное образование верхнего уровня,Алейский муниципальный район,Алейский муниципальный район,...,Муниципальный район,01601000,Без изменений,2010,2025,2009,78942.6,Тысяча рублей,Январь-март,CD


In [ ]:
# Фильтруем Москву и последний отчетный год
df = df[(df["region_name"] == "Москва") &
        (df["okved"] == "Всего") &
        (df["year"] == 2016)]

# Убираем хвостики у названий
patterns = ['Муниципальный округ ', 'Городской округ ', 'Поселение ']
for pattern in patterns:
        df['mun_district'] = df['mun_district'].str.replace(pattern, '')

# Кол-во муниципалитетов, округов и прочего
mun_num = len(df["mun_district"].unique())

print(f"Москва, 132 муниципалитета и 12 округов: {mun_num}")

Москва, 132 муниципалитета и 12 округов: 124


In [ ]:
# Готово!
df = df.rename(columns={"indicator_value": "other_goods_sold"})
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,okved,region_id,region_name,mun_level,mun_district,municipality,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,other_goods_sold,indicator_unit,indicator_period,comment
27386,39,Деятельность предприятий,Y48201002,Продано товаров несобственного производства (б...,Всего,45,Москва,Муниципальное образование верхнего уровня,Богородское,Муниципальный округ Богородское,...,Внутригородская территория города федерального...,45301000,Изменен тип и название,2010,2013,2016,2755713.6,Тысяча рублей,Январь-декабрь,CD
27400,39,Деятельность предприятий,Y48201002,Продано товаров несобственного производства (б...,Всего,45,Москва,Муниципальное образование верхнего уровня,Восточное Измайлово,Муниципальный округ Восточное Измайлово,...,Внутригородская территория города федерального...,45303000,Изменен тип и название,2010,2013,2016,7229392.4,Тысяча рублей,Январь-декабрь,CD


In [ ]:
# Продолжаем составлять итоговый датасет
result = pd.merge(result, df[["mun_district", "other_goods_sold"]], on="mun_district", how="left")
result

,mun_district,indicator_value,area,pop_dense,pop_work_dense,pop_child_dense,avg_age,trade_per_pers,pers_per_bed,self_goods_sold,other_goods_sold
0,Муниципальные образования города Москвы (столи...,13104177.0,NaN,NaN,NaN,NaN,41.300748,NaN,NaN,NaN,NaN
1,Богородское,108657.0,8.65,12561.502890,6906.127168,1996.531792,41.819579,3.661145,NaN,25493879.1,2755713.6
2,Вешняки,119927.0,10.67,11239.643861,6399.437676,1560.074977,43.079131,NaN,57.712705,22482322.9,NaN
3,Восточное Измайлово,73929.0,3.85,19202.337662,11500.000000,3111.688312,39.658564,NaN,18.088818,21708295.2,7229392.4
4,Восточный,12906.0,3.20,4033.125000,2331.562500,519.687500,42.835735,NaN,368.742857,69276.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...
142,Роговское,6478.0,178.18,36.356493,18.559883,6.240880,43.121179,NaN,NaN,NaN,NaN
143,Рязановское,32215.0,40.48,795.825099,458.078063,138.166996,41.049666,NaN,495.615385,41010696.3,9959747.0
144,Сосенское,115545.0,66.81,1729.456668,986.558898,398.877414,36.707015,NaN,3301.285714,67726829.5,112169075.1
145,Филимонковское,16237.0,35.72,454.563270,245.212766,99.972004,38.732463,NaN,NaN,882499.8,NaN


### Количество организаций

Файл: data_Y49010002_112_v20241111

In [ ]:
file_path = "drive/MyDrive/masters/geo-embeddings/data/org_num_stats.csv"

df = pd.read_csv(file_path)
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,region_id,region_name,mun_level,mun_district,municipality,oktmo,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,indicator_value,indicator_unit,indicator_period,comment
0,39,Деятельность предприятий,Y49010003,Количество индивидуальных предпринимателей по ...,1,Алтайский край,Муниципальное образование верхнего уровня,Залесовский муниципальный округ,Залесовский муниципальный округ,1512000,...,Муниципальный округ,01512000,Изменен тип и название,2022,2025,2024,217,Человек,На 1 апреля,CD
1,39,Деятельность предприятий,Y49010003,Количество индивидуальных предпринимателей по ...,1,Алтайский край,Муниципальное образование верхнего уровня,Муниципальный округ город Славгород,Муниципальный округ город Славгород,1539000,...,Муниципальный округ,01539000,"Изменен тип и название, Объединение",2023,2025,2024,854,Человек,На 1 апреля,CD


In [ ]:
# Фильтруем Москву
df = df[df["region_name"] == "Москва"]

# Убираем хвостики у названий
patterns = ['Муниципальный округ ', 'Городской округ ', 'Поселение ']
for pattern in patterns:
        df['mun_district'] = df['mun_district'].str.replace(pattern, '')

# Кол-во муниципалитетов, округов и прочего
mun_num = len(df["mun_district"].unique())

print(f"Москва, 132 муниципалитета и 12 округов: {mun_num}")

Москва, 132 муниципалитета и 12 округов: 147


In [ ]:
# Готово!
df = df.rename(columns={"indicator_value": "org_num"})
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,region_id,region_name,mun_level,mun_district,municipality,oktmo,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,org_num,indicator_unit,indicator_period,comment
6541,39,Деятельность предприятий,Y49010003,Количество индивидуальных предпринимателей по ...,45,Москва,Субъект РФ,Муниципальные образования города Москвы (столи...,Муниципальные образования города Москвы (столи...,45000000,...,Субъект РФ,CD,CD,CD,CD,2024,448529,Человек,На 1 апреля,mun_type в исходных данных не соответствует oktmo
6542,39,Деятельность предприятий,Y49010003,Количество индивидуальных предпринимателей по ...,45,Москва,Муниципальное образование верхнего уровня,Богородское,Муниципальный округ Богородское,45301000,...,Внутригородская территория города федерального...,45301000,Изменен тип и название,2010,2013,2024,3810,Человек,На 1 апреля,CD


In [ ]:
# Продолжаем составлять итоговый датасет
result = pd.merge(result, df[["mun_district", "org_num"]], on="mun_district", how="left")
result

,mun_district,indicator_value,area,pop_dense,pop_work_dense,pop_child_dense,avg_age,trade_per_pers,pers_per_bed,self_goods_sold,other_goods_sold,org_num
0,Муниципальные образования города Москвы (столи...,13104177.0,NaN,NaN,NaN,NaN,41.300748,NaN,NaN,NaN,NaN,448529
1,Богородское,108657.0,8.65,12561.502890,6906.127168,1996.531792,41.819579,3.661145,NaN,25493879.1,2755713.6,3810
2,Вешняки,119927.0,10.67,11239.643861,6399.437676,1560.074977,43.079131,NaN,57.712705,22482322.9,NaN,3397
3,Восточное Измайлово,73929.0,3.85,19202.337662,11500.000000,3111.688312,39.658564,NaN,18.088818,21708295.2,7229392.4,2008
4,Восточный,12906.0,3.20,4033.125000,2331.562500,519.687500,42.835735,NaN,368.742857,69276.7,NaN,313
...,...,...,...,...,...,...,...,...,...,...,...,...
142,Роговское,6478.0,178.18,36.356493,18.559883,6.240880,43.121179,NaN,NaN,NaN,NaN,249
143,Рязановское,32215.0,40.48,795.825099,458.078063,138.166996,41.049666,NaN,495.615385,41010696.3,9959747.0,1773
144,Сосенское,115545.0,66.81,1729.456668,986.558898,398.877414,36.707015,NaN,3301.285714,67726829.5,112169075.1,8727
145,Филимонковское,16237.0,35.72,454.563270,245.212766,99.972004,38.732463,NaN,NaN,882499.8,NaN,1171


### Количество ИП

Файл: data_Y49010003_112_v20241111

In [ ]:
file_path = "drive/MyDrive/masters/geo-embeddings/data/bus_num_stats.csv"

df = pd.read_csv(file_path)
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,region_id,region_name,mun_level,mun_district,municipality,oktmo,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,indicator_value,indicator_unit,indicator_period,comment
0,39,Деятельность предприятий,Y49010002,Количество организаций по данным государственн...,1,Алтайский край,Муниципальное образование верхнего уровня,Залесовский муниципальный округ,Залесовский муниципальный округ,1512000,...,Муниципальный округ,01512000,Изменен тип и название,2022,2025,2024,75,Единица,На 1 апреля,CD
1,39,Деятельность предприятий,Y49010002,Количество организаций по данным государственн...,1,Алтайский край,Муниципальное образование верхнего уровня,Муниципальный округ город Славгород,Муниципальный округ город Славгород,1539000,...,Муниципальный округ,01539000,"Изменен тип и название, Объединение",2023,2025,2024,266,Единица,На 1 апреля,CD


In [ ]:
# Фильтруем Москву
df = df[df["region_name"] == "Москва"]

# Убираем хвостики у названий
patterns = ['Муниципальный округ ', 'Городской округ ', 'Поселение ']
for pattern in patterns:
        df['mun_district'] = df['mun_district'].str.replace(pattern, '')

# Кол-во муниципалитетов, округов и прочего
mun_num = len(df["mun_district"].unique())

print(f"Москва, 132 муниципалитета и 12 округов: {mun_num}")

Москва, 132 муниципалитета и 12 округов: 147


In [ ]:
# Готово!
df = df.rename(columns={"indicator_value": "entrep_num"})
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,region_id,region_name,mun_level,mun_district,municipality,oktmo,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,entrep_num,indicator_unit,indicator_period,comment
6424,39,Деятельность предприятий,Y49010002,Количество организаций по данным государственн...,45,Москва,Субъект РФ,Муниципальные образования города Москвы (столи...,Муниципальные образования города Москвы (столи...,45000000,...,Субъект РФ,CD,CD,CD,CD,2024,613120,Единица,На 1 апреля,mun_type в исходных данных не соответствует oktmo
6425,39,Деятельность предприятий,Y49010002,Количество организаций по данным государственн...,45,Москва,Муниципальное образование верхнего уровня,Богородское,Муниципальный округ Богородское,45301000,...,Внутригородская территория города федерального...,45301000,Изменен тип и название,2010,2013,2024,4270,Единица,На 1 апреля,CD


In [ ]:
# Продолжаем составлять итоговый датасет
result = pd.merge(result, df[["mun_district", "entrep_num"]], on="mun_district", how="left")
result

,mun_district,indicator_value,area,pop_dense,pop_work_dense,pop_child_dense,avg_age,trade_per_pers,pers_per_bed,self_goods_sold,other_goods_sold,org_num,entrep_num
0,Муниципальные образования города Москвы (столи...,13104177.0,NaN,NaN,NaN,NaN,41.300748,NaN,NaN,NaN,NaN,448529,613120
1,Богородское,108657.0,8.65,12561.502890,6906.127168,1996.531792,41.819579,3.661145,NaN,25493879.1,2755713.6,3810,4270
2,Вешняки,119927.0,10.67,11239.643861,6399.437676,1560.074977,43.079131,NaN,57.712705,22482322.9,NaN,3397,1743
3,Восточное Измайлово,73929.0,3.85,19202.337662,11500.000000,3111.688312,39.658564,NaN,18.088818,21708295.2,7229392.4,2008,1937
4,Восточный,12906.0,3.20,4033.125000,2331.562500,519.687500,42.835735,NaN,368.742857,69276.7,NaN,313,160
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Роговское,6478.0,178.18,36.356493,18.559883,6.240880,43.121179,NaN,NaN,NaN,NaN,249,120
143,Рязановское,32215.0,40.48,795.825099,458.078063,138.166996,41.049666,NaN,495.615385,41010696.3,9959747.0,1773,598
144,Сосенское,115545.0,66.81,1729.456668,986.558898,398.877414,36.707015,NaN,3301.285714,67726829.5,112169075.1,8727,7334
145,Филимонковское,16237.0,35.72,454.563270,245.212766,99.972004,38.732463,NaN,NaN,882499.8,NaN,1171,299


## Границы административных районов

Чтобы спроецировать данные на карте - необходимо получить координаты всех муниципальных районов Москвы

In [ ]:
!wget -nc https://download.geofabrik.de/russia/central-fed-district-latest.osm.pbf

--2025-06-07 17:50:27--  https://download.geofabrik.de/russia/central-fed-district-latest.osm.pbf
Resolving download.geofabrik.de (download.geofabrik.de)... 95.217.45.61, 95.217.63.98, 95.216.245.233, ...
Connecting to download.geofabrik.de (download.geofabrik.de)|95.217.45.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 829580871 (791M) [application/octet-stream]
Saving to: ‘central-fed-district-latest.osm.pbf’

central-fed-distric 100%[===================>] 791.15M  14.0MB/s    in 57s     

2025-06-07 17:51:25 (13.9 MB/s) - ‘central-fed-district-latest.osm.pbf’ saved [829580871/829580871]



In [ ]:
overpass_url = "https://overpass-api.de/api/interpreter"
query = """
[out:json][timeout:25];
relation["boundary"="administrative"]["admin_level"="4"]["name"="Москва"];
out geom;
"""

resp = requests.get(overpass_url, params={"data": query})
data = resp.json()

In [ ]:
# Создаем список всех полигонов районов
district_polygons = []
for element in data["elements"]:
    if element["type"] == "relation":
        members = element.get("members", [])
        lines = []
        for m in members:
            if m["role"] in ["outer", ""] and "geometry" in m:  # Некоторые роли могут быть пустыми
                coords = [(pt["lon"], pt["lat"]) for pt in m["geometry"]]
                if len(coords) > 1:  # Проверяем, что линия валидна
                    lines.append(LineString(coords))

        if lines:
            try:
                # Пробуем создать полигон из линий
                polygons = list(polygonize(lines))
                if polygons:
                    district_polygon = unary_union(polygons)
                    district_polygons.append({
                        "type": "Feature",
                        "properties": {"name": element["tags"].get("name", "")},
                        "geometry": json.loads(json.dumps(district_polygon.__geo_interface__))
                    })
            except:
                print(f"Ошибка при обработке района {element.get('id')}")

# Сохраняем все районы как FeatureCollection
geojson = {
    "type": "FeatureCollection",
    "features": district_polygons
}

with open("moscow_districts.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print("Границы районов сохранены в moscow_districts.geojson")

Границы районов сохранены в moscow_districts.geojson


In [ ]:
!osmium extract \
    --overwrite \
    --output-format pbf \
    --polygon moscow_districts.geojson \
    central-fed-district-latest.osm.pbf \
    -o moscow-districts.osm.pbf

In [ ]:
fp = "moscow-districts.osm.pbf"

osm = OSM(fp)

# Получаем все административные границы
boundaries = osm.get_boundaries()

In [ ]:
print(boundaries.keys()), len(boundaries)

Index(['addr:country', 'name', 'ref', 'website', 'admin_level', 'boundary',
       'border_type', 'start_date', 'id', 'timestamp', 'version', 'changeset',
       'geometry', 'tags', 'osm_type'],
      dtype='object')


(None, 166)

In [ ]:
mun_geo = boundaries[["name", "geometry"]]
mun_geo.to_csv("mun_geo.csv")
# mun_geo = pd.read_csv("mun_geo.csv")

In [ ]:
mun_geo = mun_geo.rename(columns={"name": "mun_district"})
mun_geo['mun_district'] = mun_geo['mun_district'].str.replace("ё", 'е')

patterns = ['городской округ ', 'район ', ' район']
for pattern in patterns:
        mun_geo['mun_district'] = mun_geo['mun_district'].str.replace(pattern, '')

mun_geo.head()

,mun_district,geometry
0,Московская область,"MULTIPOLYGON (((37.04216 55.1356, 36.92123 55...."
1,Калужская область,"MULTIPOLYGON (((36.97508 55.19087, 36.92123 55..."
2,Москва,"MULTIPOLYGON (((36.80357 55.45162, 36.80451 55..."
3,Северный административный округ,"POLYGON ((37.43649 55.87958, 37.4375 55.87989,..."
4,Мытищи,"MULTIPOLYGON (((37.54074 55.94784, 37.54154 55..."


In [ ]:
replacements = {
    'Соколиная Гора': 'Соколиная гора',
    'Марьина Роща': 'Марьина роща',
    'Филевский Парк': 'Филевский парк',
    'Нагатинский Затон': 'Нагатинский затон',

}
mun_geo['mun_district'] = mun_geo['mun_district'].replace(replacements)

In [ ]:
# Добавляем координаты в итоговый датасет
final = pd.merge(result, mun_geo[["mun_district", "geometry"]], on="mun_district", how="left")
final = final.drop("indicator_value", axis=1)
final

,mun_district,area,pop_dense,pop_work_dense,pop_child_dense,avg_age,trade_per_pers,pers_per_bed,self_goods_sold,other_goods_sold,org_num,entrep_num,geometry
0,Муниципальные образования города Москвы (столи...,NaN,NaN,NaN,NaN,41.300748,NaN,NaN,NaN,NaN,448529,613120,None
1,Богородское,8.65,12561.502890,6906.127168,1996.531792,41.819579,3.661145,NaN,25493879.1,2755713.6,3810,4270,"POLYGON ((37.67021 55.83874, 37.67023 55.83865..."
2,Вешняки,10.67,11239.643861,6399.437676,1560.074977,43.079131,NaN,57.712705,22482322.9,NaN,3397,1743,"POLYGON ((37.76752 55.73415, 37.76927 55.73325..."
3,Восточное Измайлово,3.85,19202.337662,11500.000000,3111.688312,39.658564,NaN,18.088818,21708295.2,7229392.4,2008,1937,"POLYGON ((37.79943 55.79318, 37.79944 55.79311..."
4,Восточный,3.20,4033.125000,2331.562500,519.687500,42.835735,NaN,368.742857,69276.7,NaN,313,160,"MULTIPOLYGON (((37.791 56.00266, 37.79048 56.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Роговское,178.18,36.356493,18.559883,6.240880,43.121179,NaN,NaN,NaN,NaN,249,120,None
143,Рязановское,40.48,795.825099,458.078063,138.166996,41.049666,NaN,495.615385,41010696.3,9959747.0,1773,598,None
144,Сосенское,66.81,1729.456668,986.558898,398.877414,36.707015,NaN,3301.285714,67726829.5,112169075.1,8727,7334,None
145,Филимонковское,35.72,454.563270,245.212766,99.972004,38.732463,NaN,NaN,882499.8,NaN,1171,299,None


In [ ]:
# Все, что осталось - Новая Москва, так что неважно
# final[final["geometry"].isna()]
final.to_csv("mun_geo_params.csv")

## Объединение с данными ЦИАН

Даст бог процветания и долгой жизни Славе, благодаря которому этот мердж стал возможен

In [ ]:
# Скачиваем файл с диска
file_path = "drive/MyDrive/masters/geo-embeddings/data/mun_geo_params.csv"
mun_geo = pd.read_csv(file_path)
mun_geo.head(3)

In [ ]:
import json
json_path = "drive/MyDrive/masters/geo-embeddings/data/offers_moscow_wprices_51k.json"

# with open(json_path) as f:
#     flats = json.load(f)
#     print(flats)

In [ ]:
def read_concatenated_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read().strip()

    json_objects = content.split('}{')

    json_objects = [obj + '}' if i != len(json_objects) - 1 else obj for i, obj in enumerate(json_objects)]
    json_objects = ['{' + obj if i != 0 else obj for i, obj in enumerate(json_objects)]

    data = []
    for obj in json_objects:
        try:
            data.append(json.loads(obj))
        except json.JSONDecodeError as e:
            print(f"Failed to parse: {obj}\nError: {e}")

    return data

In [ ]:
list_of_dicts = read_concatenated_json(json_path)
print(f'Parsed {len(list_of_dicts)} deals.')

In [ ]:
# MULTIPOLYGON (((37.04215517788462 55.135602100 ...

# у меня есть границы районов в таком виде в колонке geometry датафрейма с муниципальными районами mun_geo. я хочу взять в датафрейме flats каждую колонку coordinated